* LRA alignment
* Author: Gavrielatos Marios
* Mayo Clinic, Fl
* Dr. Owen Ross Lab
* Last revised 09/13/2024

In [ ]:
# Import librariers
import joblib
import pandas as pd

from extract_svs import *



In [ ]:
# Input data and create tables
merged_tmp_cutesv = joblib.load(r"path\to\grch38.cutesv.merge.jl")
merged_cutesv = cutesv_extraction(merged_tmp_cutesv)
merged_tmp_sniffles2 = joblib.load(r"path\to\grch38.sniffles2.merge.jl")
merged_sniffles2 = sniffles2_extraction(merged_tmp_sniffles2)
merged_tmp_pbsv = joblib.load(r"path\to\grch38.pbsv.merge.jl")
merged_pbsv = pbsv_extraction(merged_tmp_pbsv)
merged_tmp_svdss = joblib.load(r"path\to\grch38.svdss.merge.jl")
merged_svdss = svdss_extraction(merged_tmp_svdss)

# Merge tables
list_to_merge_full=[merged_cutesv, merged_sniffles2, merged_pbsv, merged_svdss]
merged_alignment=pd.concat(list_to_merge_full)

In [ ]:
phenoSV_alignment = pd.read_csv(r"path\to\phenoSV_alignment.bed.out.csv", index_col=0).rename(columns={"ID": "id"})
pheno_dict_tmp_alignment={
    'chrom':merged_alignment['chrom'],
    'start':merged_alignment['start'],
    'end':merged_alignment['end'],
    'length':merged_alignment['svlen'],
    'id':merged_alignment.index,
    'svtype':merged_alignment['svtype'],
    'filter':merged_alignment['filter'],
    'sample_count':merged_alignment['sample_count'],
    'sample_names':merged_alignment['sample_names'],
    'genotype':merged_alignment['genotype']
}
pheno_sv_tmp = pd.DataFrame(pheno_dict_tmp_alignment)
pheno_sv_tmp = pheno_sv_tmp.reset_index(drop=True)


desired_order = [
        'chrom', 'start', 'end', 'length',
        'svtype', 'Elements', 'Type', 'filter', 'sample_count', 'sample_names', 'genotype',
        'Pathogenicity', 'Phen2Gene', 'PhenoSV', 'id'
    ]

merged_tmp1 = pd.merge(phenoSV_alignment, pheno_sv_tmp, on='id')

final_pheno_sv_df_alignment = merged_tmp1[desired_order]
final_pheno_sv_df_alignment['genotype'] = final_pheno_sv_df_alignment['genotype'].astype(str)
final_pheno_sv_df_alignment['sample_names'] = final_pheno_sv_df_alignment['sample_names'].astype(str)

final_pheno_sv_df_alignment['genotype_count'] = final_pheno_sv_df_alignment['genotype'].str.count("h")
final_pheno_sv_df_alignment = final_pheno_sv_df_alignment.sort_values('genotype_count', ascending=False).drop_duplicates(['svtype', 'Elements', 'Type', 'filter', 'sample_count', 'sample_names', 'Pathogenicity']).sort_index()



final_pheno_sv_df_alignment.to_csv(r'alignments\Alignment_full.csv', index=False)


exons = final_pheno_sv_df_alignment[final_pheno_sv_df_alignment['Type']=='Exonic']
exons.to_excel(r'alignments\Alignment_Exons.xlsx', index=False)
